# Chinese exports by year, country and commodity

### Import Python tools and Jupyter configuration

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd
import altair as alt
import glob

In [3]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None
# pd.set_option("display.float_format", lambda x: "%.2f" % x)
pd.options.display.float_format = "${:,.2f}".format

---

### Read processed trade data

In [4]:
df = pd.read_csv(
    "../data/processed/exports_xingjiang_all_countries.csv",
    dtype={
        "commodity_code": str,
        "commodity_chapter": str,
        "commodity_chapter_four": str,
    },
)

In [5]:
df = df.drop_duplicates()

---

### Exports to countries

In [6]:
xinjiang_exports = df.groupby(["exported_to"]).agg({"us_dollar": "sum"}).reset_index()

In [7]:
df[df["exported_to"] == "Kazakhstan"].to_csv(
    "../data/processed/kazakhstan.csv", index=False
)

In [8]:
xinjiang_exports.sort_values("us_dollar", ascending=False).head(20)

,exported_to,us_dollar
84,Kazakhstan,23567538140
87,Kyrgyzstan,14866972113
140,Russia,4339204979
165,Tajikistan,2919888343
181,Uzbekistan,1555308229
125,Pakistan,1103146920
179,United States,1070939568
74,India,1017187461
184,Vietnam,767979276
80,Italy,456837042


In [9]:
xinjiang_exports.to_csv(
    "../data/processed/xinjiang_exports_countries_2017_2021.csv", index=False
)

In [10]:
df.to_csv("../data/processed/xinjiang_all_countries_exports.csv", index=False)

---

### Most exported commodity types

In [69]:
df[
    [
        "commodity_chapter_desc_clean",
        "commodity_chapter_desc_clean_two",
        "trash",
        "trash",
        "trash",
        "trash",
    ]
] = df["commodity_chapter_desc"].str.split(pat=";", expand=True)

In [70]:
xinjiang_exports_types = (
    df.groupby(["commodity_chapter_desc_clean"]).agg({"us_dollar": sum}).reset_index()
)

In [71]:
xinjiang_exports_types.sort_values("us_dollar", ascending=False).head(10)

,commodity_chapter_desc_clean,us_dollar
5,Apparel and clothing accessories,19506957324
28,Footwear,8530093335
60,Plastics and articles thereof,4450636812
7,Articles of leather,2424928662
56,Organic chemicals,2136200606
20,Electrical machinery and equipment and parts thereof,2125848899
62,"Preparations of vegetables, fruit, nuts or other parts of plants",2012005412
29,"Fruit and nuts, edible",1510275242
77,"Textiles, made up articles",1494637674
36,Iron or steel articles,1378102247


In [72]:
xinjiang_exports_types.sort_values("us_dollar", ascending=False).head(10).to_csv(
    "../data/processed/xinjiang_exports_commodities_2017_2021.csv", index=False
)

### Exports by year

In [15]:
xinjiang_exports_types_year = (
    df.groupby(["year", "commodity_chapter_desc"]).agg({"us_dollar": sum}).reset_index()
)

In [16]:
xinjiang_exports_types_year_pivot = (
    xinjiang_exports_types_year.pivot_table(
        columns="year", index="commodity_chapter_desc", values="us_dollar"
    )
    .fillna(0)
    .reset_index()
)

In [17]:
xinjiang_exports_types_year_pivot.columns = (
    xinjiang_exports_types_year_pivot.columns.astype(str)
)

In [18]:
xinjiang_exports_types_year_pivot["2021"] = xinjiang_exports_types_year_pivot[
    "2021"
].round(0)

In [19]:
xinjiang_exports_types_year_pivot.head()

year,commodity_chapter_desc,2017,2018,2019,2020,2021
0,"Aircraft, spacecraft and parts thereof",$0.00,$0.00,$0.00,$0.00,"$58,259.00"
1,Albuminoidal substances; modified starches; glues; enzymes,"$9,823,866.00","$11,648,087.00","$11,273,195.00","$7,815,793.00","$8,122,353.00"
2,Aluminium and articles thereof,"$64,529,277.00","$116,933,622.00",$0.00,$0.00,"$112,382,667.00"
3,Animal or vegetable fats and oils and their cleavage products; prepared animal fats; animal or vegetable waxes,"$126,051.00","$104,069.00","$82,822.00","$48,143.00","$56,431.00"
4,Animal originated products; not elsewhere specified or included,"$17,981,235.00","$17,393,941.00","$10,039,977.00","$8,030,283.00","$10,654,213.00"


In [20]:
xinjiang_exports_types_year_pivot.sort_values("2021", ascending=False).head(10)

year,commodity_chapter_desc,2017,2018,2019,2020,2021
6,Apparel and clothing accessories; not knitted or crocheted,"$2,845,458,468.00","$2,465,034,316.00","$2,791,886,711.00",$0.00,"$4,259,143,923.00"
21,"Electrical machinery and equipment and parts thereof; sound recorders and reproducers; television image and sound recorders and reproducers, parts and accessories of such articles",$0.00,$0.00,$0.00,$0.00,"$2,125,848,899.00"
30,Footwear; gaiters and the like; parts of such articles,"$2,890,864,457.00","$1,958,535,325.00","$1,865,182,639.00",$0.00,"$1,815,510,914.00"
5,Apparel and clothing accessories; knitted or crocheted,"$1,897,409,309.00","$1,540,648,501.00","$1,773,908,024.00","$187,337,449.00","$1,746,130,623.00"
62,Plastics and articles thereof,"$720,671,878.00","$747,512,043.00","$760,309,024.00","$863,236,970.00","$1,358,906,897.00"
54,"Nuclear reactors, boilers, machinery and mechanical appliances; parts thereof",$0.00,$0.00,$0.00,$0.00,"$1,016,381,537.00"
89,"Vehicles; other than railway or tramway rolling stock, and parts and accessories thereof",$0.00,$0.00,$0.00,$0.00,"$782,275,096.00"
83,"Toys, games and sports requisites; parts and accessories thereof",$0.00,$0.00,$0.00,$0.00,"$767,135,447.00"
32,"Furniture; bedding, mattresses, mattress supports, cushions and similar stuffed furnishings; lamps and lighting fittings, n.e.c.; illuminated signs, illuminated name-plates and the like; prefabricated buildings",$0.00,$0.00,$0.00,$0.00,"$642,189,079.00"
58,Organic chemicals,"$268,769,145.00","$397,794,487.00","$479,526,675.00","$384,541,305.00","$605,568,994.00"


In [21]:
xinjiang_exports_types_year_pivot["mean"] = xinjiang_exports_types_year_pivot[
    ["2017", "2018", "2019", "2020"]
].mean(axis=1)

In [22]:
xinjiang_exports_types_year_pivot["change"] = round(
    (
        xinjiang_exports_types_year_pivot["2021"]
        - xinjiang_exports_types_year_pivot["mean"]
    )
    / xinjiang_exports_types_year_pivot["mean"]
    * 100
)

In [23]:
xinjiang_exports_types_year_pivot[
    xinjiang_exports_types_year_pivot["mean"] > 0
].sort_values("2021", ascending=False).head(10)

year,commodity_chapter_desc,2017,2018,2019,2020,2021,mean,change
6,Apparel and clothing accessories; not knitted or crocheted,"$2,845,458,468.00","$2,465,034,316.00","$2,791,886,711.00",$0.00,"$4,259,143,923.00","$2,025,594,873.75",$110.00
30,Footwear; gaiters and the like; parts of such articles,"$2,890,864,457.00","$1,958,535,325.00","$1,865,182,639.00",$0.00,"$1,815,510,914.00","$1,678,645,605.25",$8.00
5,Apparel and clothing accessories; knitted or crocheted,"$1,897,409,309.00","$1,540,648,501.00","$1,773,908,024.00","$187,337,449.00","$1,746,130,623.00","$1,349,825,820.75",$29.00
62,Plastics and articles thereof,"$720,671,878.00","$747,512,043.00","$760,309,024.00","$863,236,970.00","$1,358,906,897.00","$772,932,478.75",$76.00
58,Organic chemicals,"$268,769,145.00","$397,794,487.00","$479,526,675.00","$384,541,305.00","$605,568,994.00","$382,657,903.00",$58.00
38,Iron or steel articles,"$350,163,622.00","$466,211,827.00","$132,305,002.00",$0.00,"$429,421,796.00","$237,170,112.75",$81.00
8,"Articles of leather; saddlery and harness; travel goods, handbags and similar containers; articles of animal gut (other than silk-worm gut)","$593,675,210.00","$542,374,254.00","$481,571,958.00","$389,440,790.00","$417,866,450.00","$501,765,553.00",$-17.00
37,Iron and steel,"$301,944,321.00","$281,736,935.00","$301,229,891.00",$0.00,"$365,289,725.00","$221,227,786.75",$65.00
64,"Preparations of vegetables, fruit, nuts or other parts of plants","$390,828,456.00","$412,565,467.00","$475,350,587.00","$389,791,795.00","$343,469,107.00","$417,134,076.25",$-18.00
31,"Fruit and nuts, edible; peel of citrus fruit or melons","$306,523,578.00","$291,505,857.00","$391,548,409.00","$278,683,865.00","$242,013,533.00","$317,065,427.25",$-24.00


### Exports by year

In [24]:
agg_exports_by_year = df.groupby(["year"]).agg({"us_dollar": "sum"}).reset_index()

In [25]:
agg_exports_by_year.us_dollar.pct_change() * 100

0       NaN
1   $-12.39
2     $1.94
3   $-69.33
4   $435.32
Name: us_dollar, dtype: float64

In [26]:
agg_exports_by_year.head()

,year,us_dollar
0,2017,13436995662
1,2018,11771866361
2,2019,12000016602
3,2020,3680144438
4,2021,19700596110


In [27]:
agg_exports_by_year.to_csv(
    "../data/processed/xinjiang_exports_agg_years_2017_2021.csv", index=False
)

---

### Cotton exports

In [28]:
xinjiang_52_exports = (
    df[df["commodity_chapter"] == "52"]
    .groupby(["commodity_chapter_desc", "exported_to"])
    .agg({"us_dollar": sum})
    .reset_index()
)

In [29]:
xinjiang_52_exports.sort_values("us_dollar", ascending=False).head(20)

,commodity_chapter_desc,exported_to,us_dollar
26,Cotton,Kyrgyzstan,99004238
25,Cotton,Kazakhstan,61101229
66,Cotton,Vietnam,29191946
39,Cotton,Pakistan,22047789
59,Cotton,Tajikistan,16899388
24,Cotton,Italy,6559469
65,Cotton,Uzbekistan,5733623
4,Cotton,Bangladesh,5026244
62,Cotton,United Arab Emirates,3830263
56,Cotton,Spain,2470699


### Tomatoes

In [30]:
xinjiang_0702_exports = (
    df[df["commodity_chapter_four"] == "0702"]
    .groupby(["commodity_chapter_four_desc", "exported_to"])
    .agg({"us_dollar": sum})
    .reset_index()
)

In [31]:
xinjiang_0702_exports.sort_values("us_dollar", ascending=False).head()

,commodity_chapter_four_desc,exported_to,us_dollar
0,Tomatoes; fresh or chilled,Kazakhstan,21672786
3,Tomatoes; fresh or chilled,Russia,8842666
1,Tomatoes; fresh or chilled,Kyrgyzstan,3816516
4,Tomatoes; fresh or chilled,Uzbekistan,122927
2,Tomatoes; fresh or chilled,Mongolia,4124


### Polysilicon

In [32]:
xinjiang_280461_exports = (
    df[df["commodity_code"].str.contains("280461")]
    .groupby(["year", "commodity_chapter_four_desc", "exported_to"])
    .agg({"us_dollar": sum})
    .reset_index()
)

In [33]:
xinjiang_280461_exports

,year,commodity_chapter_four_desc,exported_to,us_dollar
0,2017,"Hydrogen, rare gases and other non-metals",Pakistan,5893160
1,2019,"Hydrogen, rare gases and other non-metals",Italy,4854
2,2020,"Hydrogen, rare gases and other non-metals",Belgium,494
3,2020,"Hydrogen, rare gases and other non-metals",Croatia,1
4,2020,"Hydrogen, rare gases and other non-metals",Reunion,2
